In [12]:
# Импорт необходимых библиотек
import os  # Работа с операционной системой (пути, файлы и т.д.)
from mlagents_envs.environment import UnityEnvironment  # Основной класс для взаимодействия с Unity средой
from mlagents_envs.side_channel.engine_configuration_channel import EngineConfigurationChannel  # Канал для настройки параметров среды (скорость, качество и т.д.)
from mlagents_envs.registry import default_registry  # Реестр стандартных сред Unity (необязательно, если используете свою среду)
import numpy as np  # Работа с массивами и матрицами
import matplotlib.pyplot as plt  # Визуализация данных (графики, диаграммы)

import torch  # Основная библиотека для работы с нейросетями и тензорами
import torch.nn as nn  # Модуль для создания нейронных сетей
import torch.optim as optim  # Оптимизаторы для обучения нейросетей
import numpy as np  # Повторный импорт numpy (можно удалить, если выше уже импортировано)
from mlagents_envs.environment import UnityEnvironment  # Повторный импорт UnityEnvironment (можно удалить, если выше уже импортировано)
from mlagents_envs.base_env import ActionTuple  # Класс для упаковки действий агента

# Обучение агента в Unity ML-Agents
В этом ноутбуке показан пример запуска обучения агента в пользовательской среде Unity, расположенной в этой папке. Для этого используется пакет `mlagents` и Python API.

In [13]:
# Путь к вашему Unity окружению (укажите путь к .exe, если экспортировано, либо используйте Editor для запуска из редактора)
# env_path = None  # Если среда запускается из редактора Unity, иначе укажите путь к .exe

env_path = os.path.join(os.getcwd(), r'N:\MyRL\My_First_NPC\MyfirstMPC\UnityEnvironment.exe')  # Замените на ваш путь к .exe


engine_channel = EngineConfigurationChannel()  # Создаем канал для настройки параметров среды
env = UnityEnvironment(file_name=env_path, side_channels=[engine_channel])  # Инициализируем среду Unity с указанным .exe и каналом
env.reset()  # Сброс среды (начало нового эпизода)
# Получение информации о среде
behavior_names = list(env.behavior_specs.keys())  # Получаем список всех доступных поведений (behavior)
print('Доступные поведения:', behavior_names)  # Выводим список доступных поведений

Доступные поведения: ['MyAgent?team=0']


In [3]:
# Инициализация среды Unity
# env = UnityEnvironment(file_name="path/to/your/environment")  # Укажите путь к вашему файлу среды
# env.reset()  # Сброс среды (начало нового эпизода)

In [14]:
# Пример цикла взаимодействия с агентом
behavior_name = behavior_names[0]  # Используем первое поведение из списка
spec = env.behavior_specs[behavior_name]  # Получаем спецификацию поведения
decision_steps, terminal_steps = env.get_steps(behavior_name)  # Получаем текущие шаги агента (решения и терминальные)
for episode in range(1):  # Запускаем один эпизод
    env.reset()  # Сброс среды перед началом эпизода
    decision_steps, terminal_steps = env.get_steps(behavior_name)  # Получаем шаги после сброса
    while len(terminal_steps) == 0:  # Пока эпизод не завершён
        action = spec.action_spec.random_action(len(decision_steps))  # Генерируем случайное действие для каждого агента
        env.set_actions(behavior_name, action)  # Передаем действие в среду
        env.step()  # Делаем шаг среды
        decision_steps, terminal_steps = env.get_steps(behavior_name)  # Получаем новые шаги
    print(f"Эпизод {episode+1} завершён.")  # Сообщаем о завершении эпизода

Эпизод 1 завершён.


In [15]:
# Получение имени поведения
behavior_names = list(env.behavior_specs.keys())  # Получаем список всех поведений
behavior_name = behavior_names[0]  # Выбираем первое поведение
spec = env.behavior_specs[behavior_name]  # Получаем спецификацию выбранного поведения

In [16]:
print(f"Имя поведения: {behavior_name}")  # Выводим имя выбранного поведения
# Получение спецификации поведения
print(f"Спецификация поведения: {spec}")  # Выводим спецификацию поведения

Имя поведения: MyAgent?team=0
Спецификация поведения: BehaviorSpec(observation_specs=[ObservationSpec(shape=(6,), dimension_property=(<DimensionProperty.NONE: 1>,), observation_type=<ObservationType.DEFAULT: 0>, name='VectorSensor_size6')], action_spec=ActionSpec(continuous_size=2, discrete_branches=(1,)))


In [17]:
# Пример простой нейросети для агента
class SimpleAgentNet(nn.Module):
    def __init__(self, obs_size, action_size):
        super(SimpleAgentNet, self).__init__()  # Инициализация родительского класса
        self.fc1 = nn.Linear(obs_size, 128)  # Первый полносвязный слой
        self.fc2 = nn.Linear(128, 64)  # Второй полносвязный слой
        self.fc3 = nn.Linear(64, action_size)  # Выходной слой, размер соответствует размеру действия
    
    def forward(self, x):
        # x может быть (obs_size,) или (batch, obs_size)
        if x.dim() == 1:
            x = x.unsqueeze(0)  # Добавляем размер батча, если вход одномерный
        x = torch.relu(self.fc1(x))  # Применяем ReLU к первому слою
        x = torch.relu(self.fc2(x))  # Применяем ReLU ко второму слою
        x = self.fc3(x)  # Выходной слой (логиты действий)
        return x  # Возвращаем результат

In [18]:
# Получаем размеры входа и выхода из спецификации поведения
obs_size = spec.observation_specs[0].shape[0]  # Размерность наблюдения (входа в сеть)
action_size = spec.action_spec.continuous_size  # Размерность непрерывного действия (выхода сети)

# Инициализация сети, оптимизатора и функции потерь
net = SimpleAgentNet(obs_size, action_size)  # Создаем экземпляр нейросети
optimizer = optim.Adam(net.parameters(), lr=1e-3)  # Оптимизатор Adam для обновления весов сети
loss_fn = nn.MSELoss()  # Функция потерь (среднеквадратичная ошибка)

# Гиперпараметры
num_episodes = 100000  # Количество эпизодов для обучения
gamma = 0.99  # Коэффициент дисконтирования

print(f"Размер наблюдений: {obs_size}")  # Выводим размер наблюдений
print(f"Размер действий: {action_size}")  # Выводим размер действий
print(f"Начинаем обучение на {num_episodes} эпизодов...")  # Сообщаем о начале обучения

Размер наблюдений: 6
Размер действий: 2
Начинаем обучение на 100000 эпизодов...


In [21]:
env.reset()  # Сброс среды перед обучением
env.close()  # Закрытие среды после завершения обучения

In [19]:
import torch  # Импортируем PyTorch для проверки доступности CUDA
# Проверка доступности CUDA (GPU)
print(torch.cuda.is_available())

True


# Визуальное обучение

In [ ]:
import torch  # Импортируем PyTorch для работы с нейросетями
import numpy as np  # Импортируем numpy для работы с массивами

# Основной цикл обучения
for episode in range(num_episodes):  # Для каждого эпизода
    env.reset()  # Сброс среды
    decision_steps, terminal_steps = env.get_steps(behavior_name)  # Получаем начальные шаги
    episode_reward = 0  # Суммарная награда за эпизод
    done = False  # Флаг завершения эпизода
    
    while not done:  # Пока эпизод не завершён
        obs = decision_steps.obs[0]  # Получаем наблюдения агента
        obs_tensor = torch.tensor(obs, dtype=torch.float32)  # Преобразуем наблюдения в тензор
        
        # Получаем действия от сети
        action_tensor = net(obs_tensor)  # Прогоняем наблюдения через сеть
        action_np = action_tensor.detach().numpy()  # Переводим результат в numpy-массив
        
        # Создаем ActionTuple с непрерывными и фиктивными дискретными действиями
        action_tuple = ActionTuple()  # Создаем объект для хранения действий
        action_tuple.add_continuous(action_np)  # Добавляем непрерывные действия
        
        # Добавляем фиктивные дискретные действия (нулевые)
        discrete_action_size = spec.action_spec.discrete_size  # Размерность дискретных действий
        if discrete_action_size > 0:
            discrete_actions = np.zeros((len(decision_steps), discrete_action_size), dtype=np.int32)  # Массив нулей для дискретных действий
            action_tuple.add_discrete(discrete_actions)  # Добавляем дискретные действия
        
        env.set_actions(behavior_name, action_tuple)  # Передаем действия в среду
        env.step()  # Делаем шаг среды
        
        # Получаем новые шаги
        next_decision_steps, next_terminal_steps = env.get_steps(behavior_name)  # Получаем новые шаги агента
        
        # Вычисляем награду и проверяем завершение эпизода
        if len(next_terminal_steps) > 0:  # Если эпизод завершён
            reward = next_terminal_steps.reward[0]  # Получаем награду за финальный шаг
            done = True  # Завершаем эпизод
        else:
            reward = next_decision_steps.reward[0]  # Получаем награду за текущий шаг
        
        episode_reward += reward  # Суммируем награду
        
        # Обновляем сеть
        target = torch.tensor([reward], dtype=torch.float32)  # Целевое значение для обучения
        predicted = net(obs_tensor).mean(dim=1)  # Предсказание сети (усреднённое по батчу)
        loss = loss_fn(predicted, target)  # Вычисляем ошибку
        
        optimizer.zero_grad()  # Обнуляем градиенты
        loss.backward()  # Вычисляем градиенты
        optimizer.step()  # Делаем шаг оптимизации
        
        decision_steps = next_decision_steps  # Переходим к следующим шагам
    
    if (episode + 1) % 100 == 0:  # Каждые 100 эпизодов выводим прогресс
        print(f"Эпизод {episode + 1}/{num_episodes}, награда: {episode_reward:.2f}")

# Сохранение итоговой модели в формате ONNX
dummy_input = torch.randn(1, obs_size)  # Создаем фиктивный вход для экспорта
torch.onnx.export(
    net,  # Экспортируемая модель
    dummy_input,  # Пример входа
    "trained_agent.onnx",  # Имя выходного файла
    input_names=['obs_0'],  # Имя входного тензора
    output_names=['continuous_actions'],  # Имя выходного тензора
    dynamic_axes={'obs_0': {0: 'batch_size'}, 'continuous_actions': {0: 'batch_size'}},  # Динамический размер батча
    opset_version=9  # Версия ONNX
 )

print("Модель успешно сохранена в формате ONNX: trained_agent.onnx")  # Сообщаем об успешном сохранении

Модель успешно сохранена в формате ONNX: trained_agent.onnx


In [ ]:
import os  # Импортируем модуль для работы с файлами
print("Файл существует:", os.path.exists("trained_agent.onnx"))  # Проверяем, был ли успешно сохранён файл модели

Файл существует: True


# Визуальное обучение на CUDA. 

In [ ]:
import torch  # Импортируем PyTorch для работы с нейросетями и CUDA
import numpy as np  # Импортируем numpy для работы с массивами

# Проверяем доступность CUDA
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Используемое устройство: {device}")  # Сообщаем, на чём работаем

# Переносим модель на устройство (GPU или CPU)
net = SimpleAgentNet(obs_size, action_size).to(device)  # Инициализируем модель и сразу кидаем на GPU
optimizer = optim.Adam(net.parameters(), lr=1e-3)  # Оптимизатор Adam
loss_fn = nn.MSELoss()  # Функция потерь (среднеквадратичная ошибка)

# Основной цикл обучения
for episode in range(num_episodes):  # Для каждого эпизода
    env.reset()  # Сброс среды
    decision_steps, terminal_steps = env.get_steps(behavior_name)  # Получаем начальные шаги
    episode_reward = 0  # Суммарная награда за эпизод
    done = False  # Флаг завершения эпизода
    
    while not done:  # Пока эпизод не завершён
        obs = decision_steps.obs[0]  # Получаем наблюдения агента
        obs_tensor = torch.tensor(obs, dtype=torch.float32).to(device)  # Преобразуем в тензор и кидаем на GPU
        
        # Получаем действия от сети
        action_tensor = net(obs_tensor)  # Прогоняем наблюдения через сеть на GPU
        action_np = action_tensor.detach().cpu().numpy()  # Возвращаем на CPU для работы с Unity
        
        # Создаем ActionTuple с непрерывными и фиктивными дискретными действиями
        action_tuple = ActionTuple()  # Создаем объект для хранения действий
        action_tuple.add_continuous(action_np)  # Добавляем непрерывные действия
        
        # Добавляем фиктивные дискретные действия (нулевые)
        discrete_action_size = spec.action_spec.discrete_size  # Размерность дискретных действий
        if discrete_action_size > 0:
            discrete_actions = np.zeros((len(decision_steps), discrete_action_size), dtype=np.int32)  # Массив нулей
            action_tuple.add_discrete(discrete_actions)  # Добавляем дискретные действия
        
        env.set_actions(behavior_name, action_tuple)  # Передаем действия в среду
        env.step()  # Делаем шаг среды
        
        # Получаем новые шаги
        next_decision_steps, next_terminal_steps = env.get_steps(behavior_name)  # Получаем новые шаги агента
        
        # Вычисляем награду и проверяем завершение эпизода
        if len(next_terminal_steps) > 0:  # Если эпизод завершён
            reward = next_terminal_steps.reward[0]  # Получаем награду за финальный шаг
            done = True  # Завершаем эпизод
        else:
            reward = next_decision_steps.reward[0]  # Получаем награду за текущий шаг
        
        episode_reward += reward  # Суммируем награду
        
        # Обновляем сеть
        target = torch.tensor([reward], dtype=torch.float32).to(device)  # Целевое значение на GPU
        predicted = net(obs_tensor).mean(dim=1)  # Предсказание сети (усреднённое по батчу)
        loss = loss_fn(predicted, target)  # Вычисляем ошибку
        
        optimizer.zero_grad()  # Обнуляем градиенты
        loss.backward()  # Вычисляем градиенты
        optimizer.step()  # Делаем шаг оптимизации
        
        decision_steps = next_decision_steps  # Переходим к следующим шагам
    
    if (episode + 1) % 100 == 0:  # Каждые 100 эпизодов выводим прогресс
        print(f"Эпизод {episode + 1}/{num_episodes}, награда: {episode_reward:.2f}")

# Сохранение итоговой модели в формате ONNX
dummy_input = torch.randn(1, obs_size).to(device)  # Фиктивный вход на GPU
torch.onnx.export(
    net,  # Экспортируемая модель
    dummy_input,  # Пример входа
    "trained_agent.onnx",  # Имя выходного файла
    input_names=['obs_0'],  # Имя входного тензора
    output_names=['continuous_actions'],  # Имя выходного тензора
    dynamic_axes={'obs_0': {0: 'batch_size'}, 'continuous_actions': {0: 'batch_size'}},  # Динамический размер батча
    opset_version=9  # Версия ONNX
)

print("Модель успешно сохранена в формате ONNX: trained_agent.onnx")  # Сообщаем об успешном сохранении

In [24]:
import torch  # PyTorch для нейронок и CUDA
import numpy as np  # Для работы с массивами
import time  # Для замера времени эпизодов
from mlagents_envs.environment import UnityEnvironment
from mlagents_envs.side_channel.engine_configuration_channel import EngineConfigurationChannel
from mlagents_envs.base_env import ActionTuple

# Настройка канала для ускорения Unity
engine_channel = EngineConfigurationChannel()
engine_channel.set_configuration_parameters(
    time_scale=50.0,  # Ускоряем симуляцию в 50 раз (попробуй 20, 50 или 100, если нужно ещё быстрее)
    target_frame_rate=60,  # Ограничиваем FPS для стабильности
    quality_level=0  # Минимальное качество графики для снижения нагрузки
)

# Инициализация среды Unity с рендерингом
env_path = os.path.join(os.getcwd(), r'N:\MyRL\My_First_NPC\MyfirstMPC\UnityEnvironment.exe')
env = UnityEnvironment(
    file_name=env_path,
    side_channels=[engine_channel],
    no_graphics=False  # Включаем рендеринг для визуального кайфа
)
env.reset()

# Проверяем CUDA
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Используемое устройство: {device}")
if not torch.cuda.is_available():
    print("CUDA недоступна, обучение будет на CPU!")
torch.cuda.empty_cache()

# Переносим модель на устройство
net = SimpleAgentNet(obs_size, action_size).to(device)
optimizer = optim.Adam(net.parameters(), lr=1e-3)
loss_fn = nn.MSELoss()

# Основной цикл обучения
try:
    for episode in range(5):
        env.reset()
        decision_steps, terminal_steps = env.get_steps(behavior_name)
        episode_reward = 0
        done = False
        start_time = time.time()  # Замеряем время эпизода

        print(f"Эпизод {episode + 1}: Кол-во агентов: {len(decision_steps)}, Форма наблюдений: {decision_steps.obs[0].shape}")

        while not done:
            obs = decision_steps.obs[0]  # Наблюдения всех агентов
            obs_tensor = torch.tensor(obs, dtype=torch.float32).to(device)  # На GPU

            # Получаем действия
            action_tensor = net(obs_tensor)
            action_np = action_tensor.detach().cpu().numpy()

            # ActionTuple для всех агентов
            action_tuple = ActionTuple()
            action_tuple.add_continuous(action_np)

            # Добавляем фиктивные дискретные действия
            discrete_action_size = spec.action_spec.discrete_size
            if discrete_action_size > 0:
                discrete_actions = np.zeros((len(decision_steps), discrete_action_size), dtype=np.int32)
                action_tuple.add_discrete(discrete_actions)

            env.set_actions(behavior_name, action_tuple)
            env.step()

            # Получаем новые шаги
            next_decision_steps, next_terminal_steps = env.get_steps(behavior_name)

            # Вычисляем награду и проверяем завершение
            if len(next_terminal_steps) > 0:
                reward = next_terminal_steps.reward[0]
                done = True
            else:
                reward = next_decision_steps.reward[0]

            episode_reward += reward

            # Обновляем сеть
            target = torch.tensor([reward], dtype=torch.float32).to(device)
            predicted = net(obs_tensor).mean(dim=1)
            loss = loss_fn(predicted, target)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            decision_steps = next_decision_steps

        episode_time = time.time() - start_time
        print(f"Эпизод {episode + 1}/{num_episodes}, Награда: {episode_reward:.2f}, Время: {episode_time:.2f} сек")

    # Сохранение модели (как в предыдущем ответе)
    dummy_input = torch.randn(1, obs_size).to(device)
    torch.onnx.export(
        net,
        dummy_input,
        "trained_agent.onnx",
        input_names=['obs_0'],
        output_names=['continuous_actions'],
        dynamic_axes={'obs_0': {0: 'batch_size'}, 'continuous_actions': {0: 'batch_size'}},
        opset_version=9,
        do_constant_folding=True
    )

    # Добавляем метаданные вручную
    import onnx
    from onnx import helper

    model = onnx.load("trained_agent.onnx")
    metadata_props = {
        "version_number": "1.2.0",
        "behavior_name": behavior_name,
        "action_size": str(action_size),
        "observation_size": str(obs_size),
        "discrete_branches": str(spec.action_spec.discrete_branches),
        "continuous_size": str(spec.action_spec.continuous_size)
    }

    for key, value in metadata_props.items():
        meta = model.metadata_props.add()
        meta.key = key
        meta.value = value

    onnx.save(model, "trained_agent.onnx")
    print("Модель успешно сохранена в формате ONNX с метаданными")

except Exception as e:
    print(f"Ошибка в цикле обучения: {e}")
    raise e

finally:
    env.close()  # Закрываем среду

Используемое устройство: cuda
Эпизод 1: Кол-во агентов: 1, Форма наблюдений: (1, 6)
Эпизод 1/10, Награда: -7.32, Время: 0.56 сек
Эпизод 2: Кол-во агентов: 1, Форма наблюдений: (1, 6)
Эпизод 2/10, Награда: -7.08, Время: 0.68 сек
Эпизод 3: Кол-во агентов: 1, Форма наблюдений: (1, 6)
Эпизод 3/10, Награда: -6.38, Время: 0.58 сек
Эпизод 4: Кол-во агентов: 1, Форма наблюдений: (1, 6)
Эпизод 4/10, Награда: -7.54, Время: 0.81 сек
Эпизод 5: Кол-во агентов: 1, Форма наблюдений: (1, 6)
Эпизод 5/10, Награда: -5.98, Время: 0.49 сек
Модель успешно сохранена в формате ONNX с метаданными


In [1]:
import onnx

# --- ШАГ 1: Определяем точные параметры из вашего скриншота ---

# Путь к вашей модели
model_path = "trained_agent.onnx" 
# Имя поведения из Unity
behavior_name = "MyAgent" 
# Space Size из Vector Observation
observation_size = 6 
# Continuous Actions
continuous_action_size = 2 
# Discrete Branch Sizes (у вас одна ветка размером 1)
discrete_branch_sizes = (1,) 

# --- ШАГ 2: Запускаем скрипт для добавления ВСЕХ метаданных ---

print(f"Загрузка модели: {model_path}")
model = onnx.load(model_path)

# Полный набор метаданных, необходимый для ML-Agents
# "version_number": "3.0.0" - это стандартное значение для современных версий ML-Agents.
# "action_spec_version": "2" - также стандартное значение.
metadata = {
    "version_number": "3.0.0",
    "action_spec_version": "2",
    "behavior_name": behavior_name,
    "is_continuous_control": "True" if continuous_action_size > 0 else "False",
    "is_discrete_control": "True" if len(discrete_branch_sizes) > 0 and discrete_branch_sizes[0] > 0 else "False",
    # Формируем строку для action_output_shape. Например: (2, 1) -> "(2,1)"
    "action_output_shape": str( (continuous_action_size + sum(discrete_branch_sizes),) ).replace(" ", ""),
    # Другие важные поля
    "observation_space_size": str(observation_size),
    "memory_size": "0" # Установите другое значение, если используете LSTM (память)
}

print("Добавляемые метаданные:")
for key, value in metadata.items():
    print(f"  {key}: {value}")
    # Удаляем старый ключ, если он существует, чтобы избежать дубликатов
    for i in reversed(range(len(model.metadata_props))):
        if model.metadata_props[i].key == key:
            del model.metadata_props[i]

    # Добавляем новый ключ
    meta = model.metadata_props.add()
    meta.key = key
    meta.value = value

# Сохраняем обновленную модель
onnx.save(model, model_path)
print(f"\nМодель успешно сохранена в '{model_path}' со всеми необходимыми метаданными.")

Загрузка модели: trained_agent.onnx
Добавляемые метаданные:
  version_number: 3.0.0
  action_spec_version: 2
  behavior_name: MyAgent
  is_continuous_control: True
  is_discrete_control: True
  action_output_shape: (3,)
  observation_space_size: 6
  memory_size: 0

Модель успешно сохранена в 'trained_agent.onnx' со всеми необходимыми метаданными.


# Обучение без рендеринга видео. 